# Kepler Framework, Emulate v12

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy

import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...

Applying ATLAS style settings...

Applying ATLAS style settings...


In [2]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [3]:
dpath_data17 = '../../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = '../../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_mc16   = '../../data/output/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.dataframe.h5'

In [4]:
tpath_data17 = 'output/emulation/data17_table_v12.h5'
tpath_data18 = 'output/emulation/data18_table_v12.h5'
tpath_mc16   = 'output/emulation/mc16_table_v12.h5'

## Setup Chains:

In [5]:
# create my chain
chains = [
            Chain( "HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI", L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v12_vloose'),
            Chain( "HLT_e28_lhtight_nod0_ringer_v12_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v12_tight' ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v12_medium'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v12" , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_v12_loose'),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'ringer_v12_vloose')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'ringer_v12_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v12_medium')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v12_loose')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [6]:
data17_df = pd.concat( (load_hdf(dpath_data17), load_hdf(tpath_data17)), axis=1)

In [7]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [02:02<00:00, 30.58s/it]


In [8]:
data17_df.shape

(43311283, 111)

### Validate:

In [9]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_v12.root')

2022-03-12 22:11:32,226 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_v12.root


In [10]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v12_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v12'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [02:01<00:00, 24.37s/it]


HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.5258  | 29972729/32047563 |
|   L2   | 93.2891  | 29896879/32047563 |
| EFCalo | 91.9759  | 29476044/32047563 |
|  HLT   | 90.0286  | 28851985/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:26<00:00, 17.40s/it]


HLT_e28_lhtight_nod0_ringer_v12_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 94.2296  | 21451925/22765602 |
|   L2   | 94.1167  | 21426229/22765602 |
| EFCalo | 93.1920  | 21215713/22765602 |
|  HLT   | 87.0266  | 19812124/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.18s/it]


HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 97.4190  | 1332484/1367786 |
|   L2   | 97.3300  | 1331266/1367786 |
| EFCalo | 95.2400  | 1302679/1367786 |
|  HLT   | 91.9559  | 1257760/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.28it/s]

HLT_e140_lhloose_nod0_ringer_v12
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 98.0724  | 59374/60541  |
|   L2   | 98.0030  | 59332/60541  |
| EFCalo | 96.6783  | 58530/60541  |
|  HLT   | 94.7028  | 57334/60541  |
+--------+----------+--------------+


In [11]:
eff_data17_Zee.save()

In [12]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_v12.root')

2022-03-12 22:16:14,208 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_v12.root


In [13]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI', pidname='!el_lhvloose',reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v12_ivarloose' , pidname='!el_lhvloose',reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI', pidname='!el_lhvloose',reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v12'          , pidname='!el_lhvloose',reset=True)

Filling...: 100%|██████████| 5/5 [00:21<00:00,  4.36s/it]


HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo |  2.2751  | 235470/10349992  |
|   L2   |  2.1939  | 227064/10349992  |
| EFCalo |  1.9797  | 204895/10349992  |
|  HLT   |  0.1569  |  16236/10349992  |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:19<00:00,  3.93s/it]


HLT_e28_lhtight_nod0_ringer_v12_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo |  1.3860  |  128264/9254300 |
|   L2   |  1.3292  |  123011/9254300 |
| EFCalo |  1.1341  |  104952/9254300 |
|  HLT   |  0.0022  |   199/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.35s/it]


HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo |  1.5870  |  124617/7852181 |
|   L2   |  1.5197  |  119327/7852181 |
| EFCalo |  1.2844  |  100854/7852181 |
|  HLT   |  0.0034  |   267/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:12<00:00,  2.49s/it]

HLT_e140_lhloose_nod0_ringer_v12
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo |  2.4542  |  138268/5634041 |
|   L2   |  2.3361  |  131618/5634041 |
| EFCalo |  2.0340  |  114596/5634041 |
|  HLT   |  0.0423  |   2383/5634041  |
+--------+----------+-----------------+


In [14]:
eff_data17_JF17.save()

In [15]:
del data17_df

## Emulate 2018:

In [16]:
data18_df = pd.concat( (load_hdf(dpath_data18), load_hdf(tpath_data18)), axis=1)

In [17]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:36<00:00, 24.20s/it]


In [18]:
data18_df.shape

(41873973, 111)

In [19]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_v12.root')

2022-03-12 22:22:18,366 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_v12.root


In [20]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v12_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v12'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:56<00:00, 23.29s/it]


HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.5744  | 30524429/32620495 |
|   L2   | 93.3471  | 30450288/32620495 |
| EFCalo | 92.0108  | 30014386/32620495 |
|  HLT   | 90.5629  | 29542070/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:22<00:00, 16.47s/it]


HLT_e28_lhtight_nod0_ringer_v12_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 94.2032  | 21962376/23313826 |
|   L2   | 94.1087  | 21940337/23313826 |
| EFCalo | 93.3092  | 21753947/23313826 |
|  HLT   | 87.6035  | 20423728/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.20s/it]


HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 97.4305  | 1346173/1381675 |
|   L2   | 97.3442  | 1344980/1381675 |
| EFCalo | 95.2551  | 1316116/1381675 |
|  HLT   | 92.2604  | 1274739/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.05it/s]


HLT_e140_lhloose_nod0_ringer_v12
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 98.1288  | 59259/60389  |
|   L2   | 98.0311  | 59200/60389  |
| EFCalo | 96.8471  | 58485/60389  |
|  HLT   | 94.9991  | 57369/60389  |
+--------+----------+--------------+


In [21]:
eff_data18_Zee.save()

In [22]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_v12.root')

2022-03-12 22:26:47,685 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_v12.root


In [23]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI', pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v12_ivarloose' , pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI', pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v12'          , pidname='!el_lhvloose',reset=True)

Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.57s/it]


HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo |  2.7154  |  229916/8467098 |
|   L2   |  2.6295  |  222642/8467098 |
| EFCalo |  2.4214  |  205022/8467098 |
|  HLT   |  0.6701  |  56734/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.21s/it]


HLT_e28_lhtight_nod0_ringer_v12_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo |  1.8195  |  138807/7628804 |
|   L2   |  1.7647  |  134623/7628804 |
| EFCalo |  1.5771  |  120313/7628804 |
|  HLT   |  0.0525  |   4003/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:13<00:00,  2.75s/it]


HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo |  2.0125  |  132059/6561985 |
|   L2   |  1.9467  |  127741/6561985 |
| EFCalo |  1.7236  |  113100/6561985 |
|  HLT   |  0.0676  |   4435/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:10<00:00,  2.08s/it]

HLT_e140_lhloose_nod0_ringer_v12
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo |  2.9726  |  141989/4776518 |
|   L2   |  2.8471  |  135994/4776518 |
| EFCalo |  2.5601  |  122286/4776518 |
|  HLT   |  0.3629  |  17332/4776518  |
+--------+----------+-----------------+


In [24]:
eff_data18_JF17.save()

## Evaluate mc16 boosted:

In [25]:
mc16_df = pd.concat( (load_hdf(dpath_mc16), load_hdf(tpath_mc16)), axis=1)

In [26]:
emulate(mc16_df)

Emulate...: 100%|██████████| 4/4 [00:00<00:00, 28.82it/s]


In [27]:
eff_mc16 = Efficiency( output_path+'/mc16_boosted_efficiency_v12.root')

2022-03-12 22:28:04,357 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/mc16_boosted_efficiency_v12.root


In [28]:
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v12_ivarloose' , pidname='el_lhtight' , reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v12'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [00:00<00:00, 15.90it/s]


HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.7303  | 78407/78619  |
| L2Calo | 99.2559  | 78034/78619  |
|   L2   | 99.2457  | 78026/78619  |
| EFCalo | 77.8158  | 61178/78619  |
|  HLT   | 76.7359  | 60329/78619  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 16.97it/s]


HLT_e28_lhtight_nod0_ringer_v12_ivarloose
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.8427  | 72990/73105  |
| L2Calo | 98.6882  | 72146/73105  |
|   L2   | 98.6827  | 72142/73105  |
| EFCalo | 77.7621  | 56848/73105  |
|  HLT   | 74.8184  | 54696/73105  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 16.72it/s]


HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9635  | 73911/73938  |
| L2Calo | 99.0965  | 73270/73938  |
|   L2   | 99.0911  | 73266/73938  |
| EFCalo | 77.5163  | 57314/73938  |
|  HLT   | 75.5917  | 55891/73938  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 17.93it/s]

HLT_e140_lhloose_nod0_ringer_v12
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9882  | 67947/67955  |
| L2Calo | 99.3790  | 67533/67955  |
|   L2   | 99.3746  | 67530/67955  |
| EFCalo | 77.2408  | 52489/67955  |
|  HLT   | 75.7693  | 51489/67955  |
+--------+----------+--------------+


In [29]:
eff_mc16.save()

## 